In [3]:
import base64
 
def write_data(file_name: str, data: bytes):
    data = base64.b64encode(data)
    with open(file_name, 'wb') as f: 
        f.write(data)
  
def read_data(file_name: str) -> bytes:
    with open(file_name, 'rb') as f:
        data = f.read()
    return base64.b64decode(data)


In [4]:
import tenseal as ts

context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree = 8192, coeff_mod_bit_sizes = [60, 40, 40, 60])
context.generate_galois_keys()
context.global_scale = 2**40

secret_context = context.serialize(save_secret_key = True)
write_data('secret.txt', secret_context)
  
context.make_context_public()
public_context = context.serialize()
write_data('public.txt', public_context)




NameError: name 'ts' is not defined

# Data owner encrypts salary, wage increase rate and bonus

In [ ]:
wage_weight = [1.2]
wage_weight_encrypted = ts.ckks_vector(context, wage_weight)
write_data('wage_weight_encrypted.txt', wage_weight_encrypted.serialize())
 
bonus_weight = [600]
bonus_weight_encrypted = ts.ckks_vector(context, bonus_weight)
write_data('bonus_weight_encrypted.txt', bonus_weight_encrypted.serialize())


# Data operator calculates the new salary

In [ ]:
context = ts.context_from(read_data('public.txt'))

# base salary encrypted
salary_proto = read_data('salary_encrypted.txt')
salary_encrypted = ts.lazy_ckks_vector_from(salary_proto)
salary_encrypted.link_context(context)
 
# wage increase percentage encrypted
w_proto = read_data('wage_weight_encrypted.txt')
w_encrypted = ts.lazy_ckks_vector_from(w_proto)
w_encrypted.link_context(context)
 
# bonus encrypted
b_proto = read_data('bonus_weight_encrypted.txt')
b_encrypted = ts.lazy_ckks_vector_from(b_proto)
b_encrypted.link_context(context)

new_salary_encrypted = ( salary_encrypted * w_encrypted ) + b_encrypted
write_data('new_salary_encrypted.txt', new_salary_encrypted)

w = ts.plain_vector([1.2])
b = ts.plain_vector([600])
 
new_salary_encrypted_v2 = ( salary_encrypted * w ) + b
write_data('new_salary_encrypted_v2.txt', new_salary_encrypted_v2)

# Data owner decrypts new salary

In [ ]:
m1_proto = read_data('new_salary_encrypted.txt')
m1 = ts.lazy_ckks_vector_from(m1_proto)
m1.link_context(context)
print(round(m1.decrypt()[0], 2))
 
m2_proto = read_data('new_salary_encrypted_v2.txt')
m2 = ts.lazy_ckks_vector_from(m2_proto)
m2.link_context(context)
print(round(m2.decrypt()[0], 2))